In [631]:
import numpy as np
import plotly as py
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [632]:
# read csv file
bt = pd.read_csv('bt.csv', sep=';')	
sp = pd.read_csv('sp.csv', sep=';')
lu = pd.read_csv('lu.csv', sep=';')

# Data treatment

In [633]:
bt.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,bt-mz.D.8,00:02:25,00:02:24,00:02:25
1,4,12,16,bt-mz.D.12,00:01:52,00:01:52,00:01:52
2,4,16,12,bt-mz.D.16,00:01:51,00:01:50,00:01:50
3,4,24,8,bt-mz.D.24,00:01:39,00:01:39,00:01:39
4,4,32,6,bt-mz.D.32,00:01:46,00:01:45,00:01:45


In [634]:
sp.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,sp-mz.D.8,00:01:29,00:01:30,00:01:30
1,4,12,16,sp-mz.D.12,00:01:22,00:01:20,00:01:22
2,4,16,12,sp-mz.D.16,00:01:06,00:01:07,00:01:08
3,4,24,8,sp-mz.D.24,00:00:59,00:00:57,00:00:57
4,4,32,6,sp-mz.D.32,00:00:58,00:00:58,00:00:59


In [635]:
lu.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,lu-mz.D.8,00:02:31,00:02:30,00:02:32
1,4,12,16,lu-mz.D.12,00:23:33,00:23:40,00:23:25
2,4,16,12,lu-mz.D.16,00:02:36,00:02:37,00:02:38
3,6,12,24,lu-mz.D.12,00:18:19,00:18:27,00:18:37
4,8,16,24,lu-mz.D.16,00:01:17,00:01:16,00:01:16


In [636]:
# change 'partition' column to string
bt['partition'] = bt['partition'].astype(str)
sp['partition'] = sp['partition'].astype(str)
lu['partition'] = lu['partition'].astype(str)

In [637]:
# change elapsed1, elapsed2, elapsed3 to float in seconds
for n in range(0,3):
    bt[f'elapsed{n+1}'] = bt[f'elapsed{n+1}'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))
    sp[f'elapsed{n+1}'] = sp[f'elapsed{n+1}'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))
    lu[f'elapsed{n+1}'] = lu[f'elapsed{n+1}'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))


In [638]:
bt.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,bt-mz.D.8,145,144,145
1,4,12,16,bt-mz.D.12,112,112,112
2,4,16,12,bt-mz.D.16,111,110,110
3,4,24,8,bt-mz.D.24,99,99,99
4,4,32,6,bt-mz.D.32,106,105,105


In [639]:
sp.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,sp-mz.D.8,89,90,90
1,4,12,16,sp-mz.D.12,82,80,82
2,4,16,12,sp-mz.D.16,66,67,68
3,4,24,8,sp-mz.D.24,59,57,57
4,4,32,6,sp-mz.D.32,58,58,59


In [640]:
lu.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,lu-mz.D.8,151,150,152
1,4,12,16,lu-mz.D.12,1413,1420,1405
2,4,16,12,lu-mz.D.16,156,157,158
3,6,12,24,lu-mz.D.12,1099,1107,1117
4,8,16,24,lu-mz.D.16,77,76,76


# Mean and standart deviation elapsed time

In [641]:
# add a new column to the bt dataframe with the name 'mean elapsed t' as the result of the mean of the 'elapsed1', 'elapsed2' and 'elapsed3' columns
bt['mean elapsed t'] = bt[['elapsed1', 'elapsed2', 'elapsed3']].mean(axis=1)
sp['mean elapsed t'] = sp[['elapsed1', 'elapsed2', 'elapsed3']].mean(axis=1)
lu['mean elapsed t'] = lu[['elapsed1', 'elapsed2', 'elapsed3']].mean(axis=1)
# add a new column to the bt dataframe with the name 'sd elapsed t' as the result of the standart deviation of the 'elapsed1', 'elapsed2' and 'elapsed3' columns
bt['sd elapsed t'] = bt[['elapsed1', 'elapsed2', 'elapsed3']].std(axis=1)
sp['sd elapsed t'] = sp[['elapsed1', 'elapsed2', 'elapsed3']].std(axis=1)
lu['sd elapsed t'] = lu[['elapsed1', 'elapsed2', 'elapsed3']].std(axis=1)

In [642]:
bt.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t
0,4,8,24,bt-mz.D.8,145,144,145,144.666667,0.57735
1,4,12,16,bt-mz.D.12,112,112,112,112.000000,0.00000
2,4,16,12,bt-mz.D.16,111,110,110,110.333333,0.57735
3,4,24,8,bt-mz.D.24,99,99,99,99.000000,0.00000
4,4,32,6,bt-mz.D.32,106,105,105,105.333333,0.57735


In [643]:
sp.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t
0,4,8,24,sp-mz.D.8,89,90,90,89.666667,0.577350
1,4,12,16,sp-mz.D.12,82,80,82,81.333333,1.154701
2,4,16,12,sp-mz.D.16,66,67,68,67.000000,1.000000
3,4,24,8,sp-mz.D.24,59,57,57,57.666667,1.154701
4,4,32,6,sp-mz.D.32,58,58,59,58.333333,0.577350


In [644]:
lu.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t
0,4,8,24,lu-mz.D.8,151,150,152,151.000000,1.000000
1,4,12,16,lu-mz.D.12,1413,1420,1405,1412.666667,7.505553
2,4,16,12,lu-mz.D.16,156,157,158,157.000000,1.000000
3,6,12,24,lu-mz.D.12,1099,1107,1117,1107.666667,9.018500
4,8,16,24,lu-mz.D.16,77,76,76,76.333333,0.577350


# Theoretic speedup data

In [645]:
def get_speed_up(mode):
    it = []; speed_up = []
    aux = 3
    for node in [4,6,8]:
        header = True
        file = f'./Out/{mode}-{node}-1.out'
        with open(file, 'r') as f:
            found = False
            for exe in range(8) if mode == 'bt' or mode == 'sp' else range(aux):
                # ignore the first 6 lines
                for i in range(6) if header is True else range(3):
                    f.readline()
                header = False
                # read the next line
                line = f.readline()
                # get the number after the word 'Iterations:'
                if not found:
                    l = 2 if mode == 'bt' or mode == 'lu' else 7
                    it_exe = int(line.split(' ')[l])
                    found = True
                it.append(it_exe)
                # ignore the next 5 lines
                for i in range(5):
                    f.readline()
                # read the next line
                line = f.readline()
                # get the last number in the line
                speed_up_exe = float(line.split(' ')[-1])
                speed_up.append(speed_up_exe)
                # ignore the next 75 lines
                for i in range(63):
                    f.readline()
                if mode == 'sp':
                    for i in range(13):
                        f.readline()
                elif mode == 'lu':
                    for i in range(6):
                        f.readline()
        print(f'end mode {mode} with {node} nodes')
        aux = 1
    return it, speed_up

In [646]:
# get the speed up for each mode
it_bt, speed_up_bt = get_speed_up('bt')
it_sp, speed_up_sp = get_speed_up('sp')
it_lu, speed_up_lu = get_speed_up('lu')

end mode bt with 4 nodes
end mode bt with 6 nodes
end mode bt with 8 nodes
end mode sp with 4 nodes
end mode sp with 6 nodes
end mode sp with 8 nodes
end mode lu with 4 nodes
end mode lu with 6 nodes
end mode lu with 8 nodes


In [647]:
# append the speed up to the dataframes
bt['theoretical speedup'] = speed_up_bt
sp['theoretical speedup'] = speed_up_sp
lu['theoretical speedup'] = speed_up_lu
# append the iterations to the dataframes
bt['iterations'] = it_bt
sp['iterations'] = it_sp
lu['iterations'] = it_lu

In [648]:
bt.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations
0,4,8,24,bt-mz.D.8,145,144,145,144.666667,0.57735,191.98,250
1,4,12,16,bt-mz.D.12,112,112,112,112.000000,0.00000,191.71,250
2,4,16,12,bt-mz.D.16,111,110,110,110.333333,0.57735,191.96,250
3,4,24,8,bt-mz.D.24,99,99,99,99.000000,0.00000,191.63,250
4,4,32,6,bt-mz.D.32,106,105,105,105.333333,0.57735,191.88,250


In [649]:
sp.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations
0,4,8,24,sp-mz.D.8,89,90,90,89.666667,0.577350,192.00,500
1,4,12,16,sp-mz.D.12,82,80,82,81.333333,1.154701,190.51,500
2,4,16,12,sp-mz.D.16,66,67,68,67.000000,1.000000,192.00,500
3,4,24,8,sp-mz.D.24,59,57,57,57.666667,1.154701,190.51,500
4,4,32,6,sp-mz.D.32,58,58,59,58.333333,0.577350,192.00,500


In [650]:
lu.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations
0,4,8,24,lu-mz.D.8,151,150,152,151.000000,1.000000,192.0,300
1,4,12,16,lu-mz.D.12,1413,1420,1405,1412.666667,7.505553,192.0,300
2,4,16,12,lu-mz.D.16,156,157,158,157.000000,1.000000,192.0,300
3,6,12,24,lu-mz.D.12,1099,1107,1117,1107.666667,9.018500,288.0,300
4,8,16,24,lu-mz.D.16,77,76,76,76.333333,0.577350,384.0,300


# Seedup and efficiency data

In [651]:
t1_bt = 2206
t1_sp = 1151.58
t1_lu = 1818

In [652]:
# add a column named 'empirical speedup' to the dataframes
bt['empirical speedup'] = t1_bt/bt['mean elapsed t']
sp['empirical speedup'] = t1_sp/sp['mean elapsed t']
lu['empirical speedup'] = t1_lu/lu['mean elapsed t']
# add a column named 'efficiency' to the dataframes
bt['efficiency'] = bt['empirical speedup']/(bt['cpus-per-task']*bt['ntasks'])
sp['efficiency'] = sp['empirical speedup']/(sp['cpus-per-task']*sp['ntasks'])
lu['efficiency'] = lu['empirical speedup']/(lu['cpus-per-task']*lu['ntasks'])

In [653]:
bt.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations,empirical speedup,efficiency
0,4,8,24,bt-mz.D.8,145,144,145,144.666667,0.57735,191.98,250,15.248848,0.079421
1,4,12,16,bt-mz.D.12,112,112,112,112.000000,0.00000,191.71,250,19.696429,0.102586
2,4,16,12,bt-mz.D.16,111,110,110,110.333333,0.57735,191.96,250,19.993958,0.104135
3,4,24,8,bt-mz.D.24,99,99,99,99.000000,0.00000,191.63,250,22.282828,0.116056
4,4,32,6,bt-mz.D.32,106,105,105,105.333333,0.57735,191.88,250,20.943038,0.109078


In [654]:
sp.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations,empirical speedup,efficiency
0,4,8,24,sp-mz.D.8,89,90,90,89.666667,0.577350,192.00,500,12.842900,0.066890
1,4,12,16,sp-mz.D.12,82,80,82,81.333333,1.154701,190.51,500,14.158770,0.073744
2,4,16,12,sp-mz.D.16,66,67,68,67.000000,1.000000,192.00,500,17.187761,0.089520
3,4,24,8,sp-mz.D.24,59,57,57,57.666667,1.154701,190.51,500,19.969595,0.104008
4,4,32,6,sp-mz.D.32,58,58,59,58.333333,0.577350,192.00,500,19.741371,0.102820


In [655]:
lu.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations,empirical speedup,efficiency
0,4,8,24,lu-mz.D.8,151,150,152,151.000000,1.000000,192.0,300,12.039735,0.062707
1,4,12,16,lu-mz.D.12,1413,1420,1405,1412.666667,7.505553,192.0,300,1.286928,0.006703
2,4,16,12,lu-mz.D.16,156,157,158,157.000000,1.000000,192.0,300,11.579618,0.060311
3,6,12,24,lu-mz.D.12,1099,1107,1117,1107.666667,9.018500,288.0,300,1.641288,0.005699
4,8,16,24,lu-mz.D.16,77,76,76,76.333333,0.577350,384.0,300,23.816594,0.062022


# bt application analysis

## Elapsed time vs ntasks (MPIs)

In [656]:
fig = px.line(x=bt['ntasks'], y=bt['mean elapsed t'],
                markers=True,
                color=bt['nodes'])

# change the title
fig.update_layout(title='Mean elapsed time for bt application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Elapsed ime (s)')

fig.show()

## Speedup vs ntasks (MPIs)

In [657]:
fig = px.line(x=bt['ntasks'], y=bt['empirical speedup'],
                markers=True,
                color=bt['nodes'])

# change the title
fig.update_layout(title='Mean speedup for bt application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Speedup')

fig.show()

## Efficiency vs ntasks (MPIs)

In [658]:
fig = px.line(x=bt['ntasks'], y=bt['efficiency'],
                markers=True,
                color=bt['nodes'])

# change the title
fig.update_layout(title='Mean efficiency for bt application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Efficiency')

fig.show()

# sp application analysis

## Elapsed time vs ntasks (MPIs)

In [659]:
fig = px.line(x=sp['ntasks'], y=sp['mean elapsed t'],
                markers=True,
                color=sp['nodes'])

# change the title
fig.update_layout(title='Mean elapsed time for sp application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Elapsed ime (s)')

fig.show()

## Speedup vs ntasks (MPIs)

In [660]:
fig = px.line(x=sp['ntasks'], y=sp['empirical speedup'],
                markers=True,
                color=sp['nodes'])

# change the title
fig.update_layout(title='Mean speedup for sp application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Speedup')

fig.show()

## Efficiency vs ntasks (MPIs)

In [661]:
fig = px.line(x=sp['ntasks'], y=sp['efficiency'],
                markers=True,
                color=sp['nodes'])

# change the title
fig.update_layout(title='Mean efficiency for sp application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Efficiency')

fig.show()

# lu application analysis

## Elapsed time vs ntasks (MPIs)

In [662]:
fig = px.line(x=lu['ntasks'], y=lu['mean elapsed t'],
                markers=True,
                color=lu['nodes'])

# change the title
fig.update_layout(title='Mean elapsed time for lu application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Elapsed ime (s)')

fig.show()

## Speedup vs ntasks (MPIs)

In [663]:
fig = px.line(x=lu['ntasks'], y=lu['empirical speedup'],
                markers=True,
                color=lu['nodes'])

# change the title
fig.update_layout(title='Mean speedup for lu application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Speedup')

fig.show()

## Efficiency vs ntasks (MPIs)

In [664]:
fig = px.line(x=lu['ntasks'], y=lu['efficiency'],
                markers=True,
                color=lu['nodes'])

# change the title
fig.update_layout(title='Mean efficiency for lu application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Efficiency')

fig.show()

# Economic analysis

In [665]:
# read csv file

# command to obtain a row: sacct -j <jobid> --format=JobID,CPUTime,NCPUS,ReqCPUS,ConsumedEnergy,AveCPUFreq,JobName,Elapsed -p
cost = pd.read_csv('cost.csv', sep='|')
cost.head()

,Nodes,JobID,CPUTime,NCPUS,ReqCPUS,ConsumedEnergy,AveCPUFreq,JobName,Elapsed
0,4,28024843.0,07:44:00,192,192,167.05K,154K,bt-mz.D.8,00:02:25
1,4,28024861.0,05:20:00,192,192,140.67K,1.89G,bt-mz.D.192,00:01:40
2,8,28024862.0,07:53:36,384,384,171.94K,28.29M,bt-mz.D.16,00:01:14
3,8,28024867.0,07:21:36,384,384,182.28K,5.26G,bt-mz.D.384,00:01:09
4,4,28024930.0,04:48:00,192,192,112.23K,900K,sp-mz.D.8,00:01:30


In [666]:
# transform JobName column type to string
cost['JobName'] = cost['JobName'].astype(str)

In [667]:
# add a new column named 'ntask' to the dataframes, that is the last string of the JobName column after a split
cost['ntasks'] = cost['JobName'].str.split('.').str[-1]
# trasnform ntask column type to int
cost['ntasks'] = cost['ntasks'].astype(int)

In [668]:
# print columns types
cost.dtypes

Nodes               int64
JobID             float64
CPUTime            object
NCPUS               int64
ReqCPUS             int64
ConsumedEnergy     object
AveCPUFreq         object
JobName            object
Elapsed            object
ntasks              int32
dtype: object

In [669]:
cost['Elapsed'] = cost['Elapsed'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))

In [670]:
# transform 'ConsumedEnergy' column to float. The initial format is 'x.xxK', so we need to remove the 'K' and transform to integer
cost['ConsumedEnergy'] = cost['ConsumedEnergy'].apply(lambda x: float(x[:-1]))

In [671]:
cost.dtypes

Nodes               int64
JobID             float64
CPUTime            object
NCPUS               int64
ReqCPUS             int64
ConsumedEnergy    float64
AveCPUFreq         object
JobName            object
Elapsed             int64
ntasks              int32
dtype: object

In [672]:
cost.head()

,Nodes,JobID,CPUTime,NCPUS,ReqCPUS,ConsumedEnergy,AveCPUFreq,JobName,Elapsed,ntasks
0,4,28024843.0,07:44:00,192,192,167.05,154K,bt-mz.D.8,145,8
1,4,28024861.0,05:20:00,192,192,140.67,1.89G,bt-mz.D.192,100,192
2,8,28024862.0,07:53:36,384,384,171.94,28.29M,bt-mz.D.16,74,16
3,8,28024867.0,07:21:36,384,384,182.28,5.26G,bt-mz.D.384,69,384
4,4,28024930.0,04:48:00,192,192,112.23,900K,sp-mz.D.8,90,8


In [673]:
# add a column named 'Mean power' to cost dataframe, that is the energy consumed divided by the elapsed time
cost['Mean power'] = cost['ConsumedEnergy']/cost['Elapsed']

In [674]:
cost.head()

,Nodes,JobID,CPUTime,NCPUS,ReqCPUS,ConsumedEnergy,AveCPUFreq,JobName,Elapsed,ntasks,Mean power
0,4,28024843.0,07:44:00,192,192,167.05,154K,bt-mz.D.8,145,8,1.152069
1,4,28024861.0,05:20:00,192,192,140.67,1.89G,bt-mz.D.192,100,192,1.406700
2,8,28024862.0,07:53:36,384,384,171.94,28.29M,bt-mz.D.16,74,16,2.323514
3,8,28024867.0,07:21:36,384,384,182.28,5.26G,bt-mz.D.384,69,384,2.641739
4,4,28024930.0,04:48:00,192,192,112.23,900K,sp-mz.D.8,90,8,1.247000


In [675]:
# add a new column named 'Cost' to cost dataframe, that is the mean power multiplied and divided by 3600 (to convert to hours) and multiplied by the cost of the energy (0.2)
cost['Cost'] = (cost['Mean power']*cost['Elapsed']*0.2)/3600

In [676]:
cost.head()

,Nodes,JobID,CPUTime,NCPUS,ReqCPUS,ConsumedEnergy,AveCPUFreq,JobName,Elapsed,ntasks,Mean power,Cost
0,4,28024843.0,07:44:00,192,192,167.05,154K,bt-mz.D.8,145,8,1.152069,0.009281
1,4,28024861.0,05:20:00,192,192,140.67,1.89G,bt-mz.D.192,100,192,1.406700,0.007815
2,8,28024862.0,07:53:36,384,384,171.94,28.29M,bt-mz.D.16,74,16,2.323514,0.009552
3,8,28024867.0,07:21:36,384,384,182.28,5.26G,bt-mz.D.384,69,384,2.641739,0.010127
4,4,28024930.0,04:48:00,192,192,112.23,900K,sp-mz.D.8,90,8,1.247000,0.006235


## bt application

In [677]:
# get the rows where the first string of the JobName column is 'bt' after a split
bt_cost = cost[cost['JobName'].str.split('.').str[0] == 'bt-mz']

fig = px.line(x=bt_cost['ntasks'], y=bt_cost['Mean power'],
                markers=True,
                color=bt_cost['Nodes'])

# change the title
fig.update_layout(title='Mean power for bt-cost application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Mean power (kW)')

fig.show()

## sp application

In [678]:
# get the rows where the first string of the JobName column is 'bt' after a split
sp_cost = cost[cost['JobName'].str.split('.').str[0] == 'sp-mz']

fig = px.line(x=sp_cost['ntasks'], y=sp_cost['Mean power'],
                markers=True,
                color=sp_cost['Nodes'])

# change the title
fig.update_layout(title='Mean power for sp-cost application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Mean power (kW)')

fig.show()

# Schedule lu application analysis